# 🚀 Auto Social-Media Manager (Twitter-only)
*100 % free, runs in Google Colab*

## Quick start
1. Add secrets in Colab Secrets (🔑 sidebar):
   - OPENROUTER_KEY
   - TWITTER_BEARER
   - TWITTER_CONSUMER_KEY
   - TWITTER_CONSUMER_SECRET
   - TWITTER_ACCESS_TOKEN
   - TWITTER_ACCESS_TOKEN_SECRET
2. Run all cells → leave the last cell running → tweets every 5 min.

In [ ]:
!pip install schedule tweepy requests -q

In [ ]:
import requests, schedule, time, uuid, random
from google.colab import userdata
import tweepy

In [ ]:
def generate_post(topic, platform):
    sub_topics = [
        "risk management", "support & resistance", "candlestick patterns",
        "market psychology", "position sizing", "breakout strategies",
        "bull flag vs bear flag", "stop-loss placement", "FOMO control",
        "golden cross / death cross", "volume spikes", "moving averages",
        "divergences", "price action", "crypto vs forex volatility"
    ]
    sub = random.choice(sub_topics)
    prompt = (
        f"Give a single, punchy trading tip about {sub}. "
        f"Keep it under 250 characters. Add 1-2 emojis."
    )
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {userdata.get('OPENROUTER_KEY')}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "mistralai/mistral-small-3.2-24b-instruct:free",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 60,
        "temperature": 0.75
    }
    r = requests.post(url, headers=headers, json=payload, timeout=30)
    if r.status_code != 200:
        return "Markets are closed for deep thoughts 🤔"
    text = r.json()["choices"][0]["message"]["content"].strip()
    text = text.split("\n")[0][:250]
    text += f" #{uuid.uuid4().hex[:4]}"   # avoid duplicates
    return text

In [ ]:
def post_twitter(text):
    client = tweepy.Client(
        bearer_token=userdata.get("TWITTER_BEARER"),
        consumer_key=userdata.get("TWITTER_CONSUMER_KEY"),
        consumer_secret=userdata.get("TWITTER_CONSUMER_SECRET"),
        access_token=userdata.get("TWITTER_ACCESS_TOKEN"),
        access_token_secret=userdata.get("TWITTER_ACCESS_TOKEN_SECRET")
    )
    client.create_tweet(text=text)

In [ ]:
import schedule, time
from datetime import datetime

def job():
    post_text = generate_post("trading", "Twitter")
    try:
        post_twitter(post_text)
        print(f"[{datetime.now().strftime('%H:%M')}] Posted: {post_text}")
    except Exception as e:
        print("Posting error:", e)

schedule.every(5).minutes.do(job)

print("🚀 Twitter-only trading bot started. Leave this cell running.")
while True:
    schedule.run_pending()
    time.sleep(1)